# Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [ ]:
%load_ext autoreload
%autoreload 2

from src import *
from src.setup import *
from src.process import *

### `Parameters`

In [ ]:
collection_name = "" # Define Collection name - example: "ble_12100302_Medina"
# collection_name = "Baxter_0_10_0"

### Instantiate CollectionData, Database, and JobClient objects

In [ ]:
collection = CollectionData(collection_name)
database = Database(collection)
jobclient = JobClient(collection)

### Get Working Models

In [ ]:
# Obtain models data from what is currently on disk.
models = [Model(*model) for model in collection.get_models()]

### Run `conflate_model`

In [ ]:
conflate_step_processor = ConflateModelStepProcessor(collection, models)
conflate_step_processor.execute_step(jobclient, database, timeout=20)

### Load Conflation Information to Database

In [ ]:
valid_models = conflate_step_processor.valid_entities
load_conflation(valid_models, database)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [ ]:
update_network(database)

### Get Working Reaches

In [ ]:
reaches = [Reach(*row) for row in database.get_reaches_by_models([model.id for model in valid_models])]
outlet_reaches = [reach for reach in reaches if reach.to_id is None]

### Run `extract_submodel` step

In [ ]:
submodel_step_processor = GenericReachStepProcessor(collection, reaches, "extract_submodel")
submodel_step_processor.execute_step(jobclient, database, timeout=10)

### Run `create_ras_terrain` step

In [ ]:
terrain_step_processor = GenericReachStepProcessor(
    collection, submodel_step_processor.valid_entities, "create_ras_terrain"
)
terrain_step_processor.execute_step(jobclient, database, timeout=10)

### Run `create_model_run_normal_depth` step

In [ ]:
create_model_step_processor = GenericReachStepProcessor(
    collection, terrain_step_processor.valid_entities, "create_model_run_normal_depth"
)
create_model_step_processor.execute_step(jobclient, database, timeout=15)

### Run `run_incremental_normal_depth` step

In [ ]:
nd_step_processor = GenericReachStepProcessor(
    collection, create_model_step_processor.valid_entities, "run_incremental_normal_depth"
)
nd_step_processor.execute_step(jobclient, database, timeout=25)

### Run normal depth `create_rating_curves_db`step

In [ ]:
nd_rc_step_processor = GenericReachStepProcessor(
    collection, nd_step_processor.valid_entities, "nd_create_rating_curves_db"
)
nd_rc_step_processor.execute_step(jobclient, database, timeout=10)

### Initial `run_known_wse` and Initial `create_rating_curves_db`

In [ ]:
execute_ikwse_for_network(
    outlet_reaches,
    collection,
    database,
    jobclient,
    nd_rc_step_processor.valid_entities,
    timeout=20,
)

### Final `run_known_wse`

In [ ]:
non_outlet_valid_reaches = [reach for reach in nd_rc_step_processor.valid_entities if reach.to_id is not None and reach.to_id in [valid_reach.id for valid_reach in nd_rc_step_processor.valid_entities]]
kwse_step_processor = KWSEStepProcessor(collection, non_outlet_valid_reaches)
kwse_step_processor.execute_step(jobclient, database, timeout=180)

### Final `create_rating_curves_db` step

In [ ]:
kwse_rc_step_processor = GenericReachStepProcessor(
    collection, kwse_step_processor.valid_entities, "kwse_create_rating_curves_db"
)
kwse_rc_step_processor.execute_step(jobclient, database, timeout=15)

### Merge Rating Curves

In [ ]:
load_all_rating_curves(database)

### Run `create_fim_lib` Step

In [ ]:
fimlib_step_processor = GenericReachStepProcessor(collection, nd_rc_step_processor.valid_entities, "create_fim_lib")
fimlib_step_processor.execute_step(jobclient, database, timeout=150)

### Create extent library step

In [ ]:
create_extent_lib(collection)

### Create Flows2FIM Start Reaches File

In [ ]:
create_f2f_start_file([reach.id for reach in outlet_reaches], collection.f2f_start_file)
